<a href="https://colab.research.google.com/github/franmazzaro/7506R-2C2023-GRUPO39/blob/main/7506R_TP2_GRUPO39_CHP1_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Librerias**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

# Carga de datasets

In [2]:
df_criticas = pd.read_csv('/content/sample_data/train.csv')
df_criticas_test = pd.read_csv('/content/sample_data/test.csv')


In [3]:
df_criticas.head(4)

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo


# Preprocesamiento

In [12]:
# Vectorización de texto usando CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_criticas['review_es'])
X_test = vectorizer.transform(df_criticas_test['review_es'])

# Codificar las etiquetas de sentimiento
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_criticas['sentimiento'])

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=25)

In [5]:
# Función para convertir valores
def convertir(valor):
    return "negativo" if valor == 0 else "positivo"

# Bayes Naive

Dividimos nuestro conjunto de train en 70 30

In [6]:
x_train, x_test, y_train, y_test = train_test_split(df_criticas.review_es,
                                                    df_criticas.sentimiento,
                                                    test_size=0.30,
                                                    random_state=25,
                                                    shuffle=True)

Antes de entrear un modelo para clasificacion necesitamos que este entrenamiento sea con valores numericos, y como nosotros tenemos palabras necesitamos convertirlas a vectores.

Utilizamos make_pipeline para aplicarle a nuestros modelos dos metodos distintos para vectorizar y luego el modelo de Naive Bayes Multidimensional. Una vez obtenidos los entrenamos

In [7]:
model_NB = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_NB_2 = make_pipeline(CountVectorizer(), MultinomialNB())

model_NB.fit(x_train, y_train)
model_NB_2.fit(x_train, y_train)

categorias_predichas = model_NB.predict(x_test)
categorias_predichas_2 = model_NB_2.predict(x_test)


In [8]:
print(accuracy_score(y_test, categorias_predichas))
print(accuracy_score(y_test, categorias_predichas_2))

0.8534666666666667
0.8376


Ya que el modelo 2 tuvo mejor accuracy es el que utilizamos para realizar la predicción de test

In [9]:
criticas_test = df_criticas_test['review_es']
categorias_predichas_test = model_NB.predict(criticas_test)


Generamos el archivo de predicciones

In [10]:
ids_test =  df_criticas_test.ID
filename = 'Naive_Bayes_Multidimensional'
prediccion = pd.DataFrame({"id": ids_test, "sentimiento": categorias_predichas_test})
prediccion.to_csv('/content/sample_data/prediccion_'+filename+'.csv', index = False)

# Random Forest

Optimizamos hiperparámetros

In [11]:
tree = RandomForestClassifier(random_state=1)
params_grid = {
    'bootstrap': [True, False],
    'ccp_alpha': [0.0, 0.01, 0.02],
    'class_weight': [None, 'balanced'],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [10, 50, 100, 200],
}
rand = RandomizedSearchCV(tree, params_grid, cv=10, scoring='f1', n_iter=10, random_state=1)
rand.fit(x_train, y_train)
rand.cv_results_['mean_test_score']

ValueError: ignored

In [ ]:
rand.best_params_

Usamos el mejor estimador para predecir sobre el conjunto de training y obtenemos sus métricas

In [ ]:
opt_tree = rand.best_estimator_
y_pred = opt_tree.predict(x_test)
print(classification_report(y_test, y_pred))

Finalmente predecimos sobre el conjunto de test y ajustamos la salida al formato aceptado por Kaggle

In [ ]:
categorias_predichas_test = opt_tree.predict(X_test)

# Aplicar la función a cada elemento del array usando comprensión de listas
sentimientos = [convertir(valor) for valor in categorias_predichas_test]

In [ ]:
filename = 'Random_Forest_opt'
prediccion = pd.DataFrame({"id": df_criticas_test.ID, "sentimiento": sentimientos})
prediccion.to_csv('/content/sample_data/prediccion_'+filename+'.csv', index = False)

# XG Boost

In [13]:
xg_boost_model = xgb.XGBClassifier(random_state=25, n_estimators=100)
xg_boost_model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=25, ...)

In [14]:
param_dist = {
    'learning_rate': [0.01, 0.1, 0.2, 0.3],  # Tasa de aprendizaje
    'max_depth': [3, 4, 5, 6, 7, 8, 9],  # Profundidad máxima del árbol
    'min_child_weight': [1, 2, 3, 4],  # Mínimo peso de un niño (para evitar divisiones adicionales)
    'subsample': [0.8, 0.9, 1.0],  # Proporción de muestras utilizadas para entrenar árboles
    'colsample_bytree': [0.8, 0.9, 1.0],  # Proporción de características utilizadas para entrenar árboles
    'gamma': [0, 0.1, 0.2, 0.3],  # Parámetro de regularización para la poda
    'n_estimators': [100, 200, 300],  # Número de árboles (estimadores) en el bosque
}

# Inicializa el clasificador XGBoost
xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', random_state=0)

# Realiza la búsqueda de hiperparámetros utilizando RandomizedSearchCV
n_iter = 10  # Número de combinaciones aleatorias a probar
xgb_search = RandomizedSearchCV(xgb_classifier, param_distributions=param_dist, n_iter=n_iter, scoring='f1', cv=5, random_state=0)

# Ajusta el modelo a los datos de entrenamiento
xgb_search.fit(x_train, y_train)

xgb_search.best_params_

{'subsample': 1.0,
 'n_estimators': 300,
 'min_child_weight': 4,
 'max_depth': 8,
 'learning_rate': 0.2,
 'gamma': 0.2,
 'colsample_bytree': 1.0}

In [16]:
opt_xg = xgb_search.best_estimator_
y_pred = opt_xg.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.85      0.86      7599
           1       0.85      0.88      0.86      7401

    accuracy                           0.86     15000
   macro avg       0.86      0.86      0.86     15000
weighted avg       0.86      0.86      0.86     15000



In [17]:
categorias_predichas_test_xg = opt_xg.predict(X_test)

# Aplicar la función a cada elemento del array usando comprensión de listas
sentimientos = [convertir(valor) for valor in categorias_predichas_test_xg]

In [18]:
filename = 'Xg_Boost_opt'
prediccion = pd.DataFrame({"id": df_criticas_test.ID, "sentimiento": sentimientos})
prediccion.to_csv('/content/sample_data/prediccion_'+filename+'.csv', index = False)